In [ ]:
import requests
import pandas

from bs4 import BeautifulSoup

response = requests.get("https://www.chope.co/singapore-restaurants/pages/halalrestaurantguide")
content = response.content
print(content)
soup = BeautifulSoup(content, "html.parser")

top_rest = soup.find_all("div", attrs={"id": "custom-page"})
list_tr = top_rest[0].find_all("div", attrs={"class": "col-md-4"})

list_rest = []
for tr in list_tr:
    df = {}
    df["rest_name"] = (tr.find("h2", attrs={"class": "restlinks"})).text
    hyperlink = (tr.find("a")["href"])

    response_ind = requests.get(hyperlink)
    content_ind = response_ind.content
    soup_ind = BeautifulSoup(content_ind, "html.parser")

    top_rest_ind = soup_ind.find_all("ul", attrs={"class": "col-xs-12 col-sm-6"})
    first_li = top_rest_ind[0].find_all("li")
    sec_li = top_rest_ind[1].find_all("li")
    df["cuisine"] = first_li[0].find("p").text.replace('\n', '').replace(
        '                                                  ', '')
    if len(first_li) < 5:
        df["hours"] = first_li[1].find("p").text.replace('\n', '').replace('                        ', '').replace(
            'À La Carte', '')
        df["good_for"] = first_li[3].find("p").text.replace('\n', '')
    else:
        df["hours"] = first_li[2].find("p").text.replace('\n', '').replace('                        ', '').replace(
            'À La Carte', '')
        df["good_for"] = first_li[4].find("p").text.replace('\n', '')
    df["location"] = sec_li[0].find("p").text.replace('\n', '')
    df["address"] = sec_li[1].find("p").text.replace('Map', '').replace('\n', '').replace('                        ',
                                                                                          '')

    list_rest.append(df)

print(list_rest)

df = pandas.DataFrame(list_rest)
df.to_csv("chope_halal_res.csv",index=False)
